## Natural Language processing
## Bag of words model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

In [2]:
dataset=pd.read_csv('./../0. DataSets/Restaurant_Reviews.tsv',delimiter='\t',quoting=3)

In [3]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


### Business case desta vez e um restaurante que tem reviews na coluna 1 e tem binario 0 se o cliente nao gostou e 1 se o cliente que deixou a review gostou
Vamos treinar um bag of words model para tentar prever se o comentario foi positivo ou negativo (coluna 2 e para confirmarmos a nossa analise)
Primeiro: temos que limpar a data. Limpar os comentarios

In [4]:
#Primeiro limpar so o primeiro comment para ver se estamos a ir no caminho certo

review=re.sub('[^a-zA-Z]'," ",dataset['Review'][0])  #retirar todos os numeros e sinais de pontuacao. no fundo tirar tudo o que nao for a-z
review=review.lower()  #por tudo em minusculas

review  #ver como esta a ficar

'wow    loved this place '

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords #importamos a library nltk no inicio e agora estamos a fazer o download do chamado "stopwords". Que sao as palavras que nao adicionam muita informacao a uma frase como por exemplo is, the, a, this, an etc..
#Vamos entao remover estas stopwords dos nossos comments. comecando pelo primeiro

review=review.split() #Separar a frase em palavras
review=[word for word in review if not word in set(stopwords.words('english'))]
review

[nltk_data] Downloading package stopwords to /home/xbuns/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['wow', 'loved', 'place']

In [6]:
#Agora vamos fazer stemming. que e apenas manter a raiz da palavra. por exemplo em vez de ter loves, loved.. vamos passar tudo para love
#na aula fizemos o stemming dentro do for loop anterior, mas vou fazer de forma separada para nao ficar confuso
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
review=[ps.stem(word) for word in review]
review

['wow', 'love', 'place']

In [7]:
#por a lista de strings de volta a uma so string
review=" ".join(review)
review

'wow love place'

In [8]:
#ok. agora sabemos que isto funciona para um comment. vamos fazer para todos
corpus=[]
for i in range(0,1000):
    review=re.sub('[^a-zA-Z]'," ",dataset['Review'][i])
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in set(stopwords.words('english'))]
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review]
    review=" ".join(review)
    corpus.append(review)

In [9]:
corpus[:5]

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price']

In [21]:
#Create a bag of words model
#basicamente correr todos os comentarios e por cda palavra diferente numa coluna
#se um determinado comentario (lines) tiver essa palavra por um 1 nessa coluna correspondete a essa palavra

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500) #este parametro max_fetures limita as columas a neste caso 1500 ou seja so as 1500 palavras mais frequentes
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,1].values #segunda coluna do dataset onde um 1 era um comentario positivo e o zero era negativo

#agora temos uma matrix X e um vector y podemos usar um modulo de classification. Os recommendados para natural language processing sao
#Naive Bayes, Decision tree, random forest classification

In [22]:
#COPIA do naive bayes classification exercise

from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

#feature scaling.. nao e necessario aqui porque e tudo 0 ou 1 ja
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train=sc_X.fit_transform(X_train)
# X_test=sc_X.transform(X_test)

#fitting Naive Bayes to the training set.

from sklearn.naive_bayes import GaussianNB

classifier=GaussianNB()
classifier.fit(X_train,y_train)

#Predicting the Test set results
y_pred=classifier.predict(X_test)

#Making the confusion Matrix

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true=y_test,y_pred=y_pred)

cm

array([[55, 42],
       [12, 91]])

In [23]:
#ou seja 55 previsoes positivas correctas e 91 previsoes negativas correctas
#12 42 incorrectas

(55+91)/(55+42+12+91) #accuracy.. previsoes correctas a dividir pelo total


0.73